Project 3 : 
Amazon reviews analysis. This dataset consists of a few million Amazon customer reviews (input text) and star ratings (output labels) for learning how to train fastText for sentiment analysis.


In [1]:
!mkdir train

In [2]:
import bz2

# Specify the path to the .bz2 file
file_path = '/kaggle/input/amazonreviews/test.ft.txt.bz2'

# Specify the output file name and path
output_file = '/kaggle/working/train/test.text'

# Open the .bz2 file for reading
with bz2.open(file_path, 'rb') as source_file:
    # Open the output file for writing
    with open(output_file, 'wb') as output:
        # Read and write the contents of the .bz2 file
        output.write(source_file.read())

print("Unzipping complete.")


Unzipping complete.


In [3]:
# Specify the path to the .bz2 file
file_path = '/kaggle/input/amazonreviews/train.ft.txt.bz2'

# Specify the output file name and path
output_file = '/kaggle/working/train/train.text'

# Open the .bz2 file for reading
with bz2.open(file_path, 'rb') as source_file:
    # Open the output file for writing
    with open(output_file, 'wb') as output:
        # Read and write the contents of the .bz2 file
        output.write(source_file.read())

print("Unzipping complete.")

Unzipping complete.


In [4]:
import pandas as pd
import numpy as np

In [5]:
import pandas as pd

# Specify the path to the dataset file
file_path1 = '/kaggle/working/train/train.text'

# Read the dataset file with explicit delimiter and number of columns
df = pd.read_csv(file_path, delimiter='\t', header=None, names=['Sentiment'], quoting=3, error_bad_lines=False)

# Split the 'Sentiment' column into 'Label' and 'Review' columns
df[['Label', 'Review']] = df['Sentiment'].str.split(' ', 1, expand=True)

# Remove the original 'Sentiment' column
df.drop('Sentiment', axis=1, inplace=True)

# select 10000 rows
df = df.sample(n=10000, random_state=50)

# Display the first few rows of the DataFrame
print(df.head())
print(df.shape)


/tmp/ipykernel_108/4220753502.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, delimiter='\t', header=None, names=['Sentiment'], quoting=3, error_bad_lines=False)
/tmp/ipykernel_108/4220753502.py:10: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Label', 'Review']] = df['Sentiment'].str.split(' ', 1, expand=True)


              Label                                             Review
1739578  __label__2  The Waltons complete second season: This reall...
2095450  __label__1  pitched it: It never worked right and refused ...
1886570  __label__1  Absurdity: One of the worst novels I have read...
2930083  __label__1  Doesn't do the job: This mini fridge I got to ...
498618   __label__2  Really good: I enjoyed this movie, in my opini...
(10000, 2)


In [6]:
import pandas as pd

# Specify the path to the dataset file
file_path2 = '/kaggle/working/train/test.text'

# Read the dataset file with explicit delimiter and number of columns
df2 = pd.read_csv(file_path2, delimiter='\t', header=None, names=['Sentiment'], quoting=3, error_bad_lines=False)

# Split the 'Sentiment' column into 'Label' and 'Review' columns
df2[['Label', 'Review']] = df2['Sentiment'].str.split(' ', 1, expand=True)

# Remove the original 'Sentiment' column
df2.drop('Sentiment', axis=1, inplace=True)

# select 10000 rows
df2 = df.sample(n=10000, random_state=50)

# Display the first few rows of the DataFrame
print(df2.head())
print(df2.shape)


/tmp/ipykernel_108/1592362421.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv(file_path2, delimiter='\t', header=None, names=['Sentiment'], quoting=3, error_bad_lines=False)
/tmp/ipykernel_108/1592362421.py:10: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df2[['Label', 'Review']] = df2['Sentiment'].str.split(' ', 1, expand=True)


              Label                                             Review
1368375  __label__2  Remixes of the huge hit "Dilemna": This is the...
1899682  __label__2  I havent read this one yet...: I just finished...
660486   __label__2  Good older flick: This is a very good movie wh...
3597567  __label__1  very little of value in this book: As is his s...
2633535  __label__1  Wait....Wasn't Jamie Lee Curtis suppose to be ...
(10000, 2)


In [7]:
df['Label'] = df['Label'].replace('__label__2', '2')
df['Label'] = df['Label'].replace('__label__1', '1')

In [8]:
df2['Label'] = df2['Label'].replace('__label__2', '2')
df2['Label'] = df2['Label'].replace('__label__1', '1')


In [9]:
print(df.head())

        Label                                             Review
1739578     2  The Waltons complete second season: This reall...
2095450     1  pitched it: It never worked right and refused ...
1886570     1  Absurdity: One of the worst novels I have read...
2930083     1  Doesn't do the job: This mini fridge I got to ...
498618      2  Really good: I enjoyed this movie, in my opini...


In [10]:
print(df2.head())

        Label                                             Review
1368375     2  Remixes of the huge hit "Dilemna": This is the...
1899682     2  I havent read this one yet...: I just finished...
660486      2  Good older flick: This is a very good movie wh...
3597567     1  very little of value in this book: As is his s...
2633535     1  Wait....Wasn't Jamie Lee Curtis suppose to be ...


In [11]:
import torch
from tqdm.notebook import tqdm

In [12]:
# to check the counts of the labels
df.Label.value_counts()

2    5013
1    4987
Name: Label, dtype: int64

In [13]:
# to check the counts of the labels
df2.Label.value_counts()

2    5013
1    4987
Name: Label, dtype: int64

In [14]:
set(df.Label)

{'1', '2'}

In [15]:
possible_labels=df.Label.unique()
possible_labels

array(['2', '1'], dtype=object)

In [16]:
label_dict={}
for index,possible_label in enumerate(possible_labels):
    label_dict[possible_label]=index

label_dict

{'2': 0, '1': 1}

In [17]:
df.Label=df['Label'].map(label_dict)

In [18]:
df.Label.info()

<class 'pandas.core.series.Series'>
Int64Index: 10000 entries, 1739578 to 534927
Series name: Label
Non-Null Count  Dtype
--------------  -----
10000 non-null  int64
dtypes: int64(1)
memory usage: 156.2 KB


In [19]:
df2.Label=df2['Label'].map(label_dict)
df2.Label.info()

<class 'pandas.core.series.Series'>
Int64Index: 10000 entries, 1368375 to 901069
Series name: Label
Non-Null Count  Dtype
--------------  -----
10000 non-null  int64
dtypes: int64(1)
memory usage: 156.2 KB


In [20]:
X_train=df['Review']
y_train=df['Label']

In [21]:
X_train.shape

(10000,)

In [22]:
y_train.shape

(10000,)

In [23]:
X_test=df2['Review']
y_test=df2['Label']

In [24]:
X_test.shape

(10000,)

In [25]:
y_test.shape

(10000,)

In [26]:
!pip install transformers

In [27]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [28]:
tokenizer=BertTokenizer.from_pretrained('bert-large-uncased',
                                        do_lower_case=True)

In [29]:
df['data_type'] = ['train']*df.shape[0]
df2['data_type'] = ['val']*df2.shape[0]

In [30]:
df2.head()

,Label,Review,data_type
1368375,0,"Remixes of the huge hit ""Dilemna"": This is the...",val
1899682,0,I havent read this one yet...: I just finished...,val
660486,0,Good older flick: This is a very good movie wh...,val
3597567,1,very little of value in this book: As is his s...,val
2633535,1,Wait....Wasn't Jamie Lee Curtis suppose to be ...,val


In [31]:
# Encoding train data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df['data_type'] == 'train'].Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    truncation=True,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['data_type'] == 'train'].Label.values.astype(int))

# Encoding validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df2[df2['data_type'] == 'val'].Review.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    truncation=True,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df2[df2['data_type'] == 'val'].Label.values.astype(int))


In [32]:
input_ids_val

tensor([[  101, 15193,  1997,  ...,     0,     0,     0],
        [  101,  1045,  4033,  ...,     0,     0,     0],
        [  101,  2204,  3080,  ...,     0,     0,     0],
        ...,
        [  101,  1018,  2706,  ...,     0,     0,     0],
        [  101,  7078,  6429,  ...,     0,     0,     0],
        [  101, 11113,  3676,  ...,     0,     0,     0]])

In [33]:
#TensorDataset create a single variable which stores info of inputids , attentionmsk, labels
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [34]:
len(dataset_train)

10000

In [35]:
dataset_train.tensors

(tensor([[  101,  1996, 14290,  ...,     0,     0,     0],
         [  101,  8219,  2009,  ...,     0,     0,     0],
         [  101, 18691,  3012,  ...,     0,     0,     0],
         ...,
         [  101,  2204,  2422,  ...,     0,     0,     0],
         [  101,  3319,  1997,  ...,     0,     0,     0],
         [  101, 11991,  2013,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 1, 1,  ..., 0, 1, 0]))

**Setting up BERT Pretrained model**

In [36]:
from transformers import BertForSequenceClassification

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [37]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-large-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [38]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [39]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [40]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [41]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [42]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

**Defining our Performance Metrics**

In [43]:
import numpy as np
from sklearn.metrics import f1_score

In [44]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [45]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

**Creating our Training Loop**

In [46]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [48]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [49]:
for epoch in tqdm(range(1, epochs+1)):
    model.train() #forward propagation
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward() #backwardprop
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    torch.save(model, f'BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2500 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.27917625017962416


  0%|          | 0/313 [00:00<?, ?it/s]

Validation loss: 0.09930898836879327
F1 Score (weighted): 0.9795000338250411


In [52]:
torch.save(model.state_dict(), 'Sentiment_Analysis_Amazon_Review.pt')

In [80]:
test_sentence = "The jacket fit nicely on me and the colour was also as expected. But there was a yellow stain near the front pocket of the jacket which if inspected closely looked like a food stain. This made me seriously doubt if they have given me a previously used and returned product. The stitches of some places are undone even before I tried wearing it. So I had to redo it. But I didn't return the product because its price is not so high and it is sufficient for its price."

predict_input = tokenizer.batch_encode_plus(
    [test_sentence],
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    truncation=True,
    return_tensors='pt'
)


In [81]:
predict_input

{'input_ids': tensor([[  101,  1996,  6598,  4906, 19957,  2006,  2033,  1998,  1996,  6120,
          2001,  2036,  2004,  3517,  1012,  2021,  2045,  2001,  1037,  3756,
         21101,  2379,  1996,  2392,  4979,  1997,  1996,  6598,  2029,  2065,
         20456,  4876,  2246,  2066,  1037,  2833, 21101,  1012,  2023,  2081,
          2033,  5667,  4797,  2065,  2027,  2031,  2445,  2033,  1037,  3130,
          2109,  1998,  2513,  4031,  1012,  1996, 25343,  1997,  2070,  3182,
          2024, 25757,  2130,  2077,  1045,  2699,  4147,  2009,  1012,  2061,
          1045,  2018,  2000,  2417,  2080,  2009,  1012,  2021,  1045,  2134,
          1005,  1056,  2709,  1996,  4031,  2138,  2049,  3976,  2003,  2025,
          2061,  2152,  1998,  2009,  2003,  7182,  2005,  2049,  3976,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [82]:
input_ids = predict_input['input_ids'].to(device)
attention_mask = predict_input['attention_mask'].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask)

logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

predicted_label = predictions.item()

Senti_label_dict = {'0': 'Good Review', '1': 'Bad Review'}
predicted_sentiment = Senti_label_dict.get(str(predicted_label))
print(predicted_sentiment)


Bad Review


In [84]:
# #If want to load the model
# import torch
# from transformers import BertForSequenceClassification, BertConfig

# config = BertConfig.from_pretrained('bert-large-uncased', num_labels=len(label_dict))
# model = BertForSequenceClassification.from_pretrained('bert-large-uncased', config=config)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a